In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
modelname="Resnet"
LR=0.001

In [ ]:
foldernames_to_id={"VeryMildDemented":1, "NonDemented":0, "ModerateDemented":2, "MildDemented":3}
import os
import torch

In [ ]:
from PIL import Image
from numpy import asarray
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers,models
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras import optimizers, losses
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os.path
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.1) # set validation split

In [ ]:
foldername='/content/drive/My Drive/ml_project/Dataset/'
train_images = train_datagen.flow_from_directory(
    foldername,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_images = train_datagen.flow_from_directory(
    foldername , # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 30586 images belonging to 4 classes.
Found 3398 images belonging to 4 classes.


In [ ]:
def create_model():
  mymodel = Sequential()

  pretrained_model= tf.keras.applications.resnet50.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=4,
                   weights='imagenet')
  for layer in pretrained_model.layers:
        layer.trainable=False

  mymodel.add(pretrained_model)

  mymodel.add(Flatten())
  mymodel.add(Dense(512, activation='relu'))
  mymodel.add(Dropout(0.2))
  mymodel.add(Dense(4, activation='softmax'))
  mymodel.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr=LR), metrics = ['accuracy'])
  return mymodel

In [ ]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24,638,852
Trainable params: 1,051,140
Non-trainable params: 23,587,712
_____________________________________

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
checkpointdir='/content/drive/My Drive/ml_project/Checkpoints/'
checkpoint_path = checkpointdir+modelname+"training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
batch_size = 32
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_best_only=False,
    save_weights_only=True)

In [ ]:
# Evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
history = model.fit(train_images,
    steps_per_epoch=len(train_images),
    validation_data=validation_images,
    validation_steps=len(validation_images),
    epochs=200,
    callbacks=[cp_callback])

Epoch 1/200
956/956 [==============================] - ETA: 0s - loss: 0.9577 - accuracy: 0.5550
Epoch 1: saving model to /content/drive/My Drive/ml_project/Checkpoints/Resnettraining_1/cp.ckpt
956/956 [==============================] - 5254s 5s/step - loss: 0.9577 - accuracy: 0.5550 - val_loss: 0.8540 - val_accuracy: 0.6368
Epoch 2/200
956/956 [==============================] - ETA: 0s - loss: 0.9665 - accuracy: 0.5561
Epoch 2: saving model to /content/drive/My Drive/ml_project/Checkpoints/Resnettraining_1/cp.ckpt
956/956 [==============================] - 126s 131ms/step - loss: 0.9665 - accuracy: 0.5561 - val_loss: 0.9646 - val_accuracy: 0.5727
Epoch 3/200
956/956 [==============================] - ETA: 0s - loss: 0.9573 - accuracy: 0.5572
Epoch 3: saving model to /content/drive/My Drive/ml_project/Checkpoints/Resnettraining_1/cp.ckpt
956/956 [==============================] - 126s 132ms/step - loss: 0.9573 - accuracy: 0.5572 - val_loss: 0.8523 - val_accuracy: 0.6224
Epoch 4/200
956

In [ ]:
#Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

2 following cells are for loading latest trained models

In [ ]:
# Create a basic model instance
#model = create_model()

# Evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# Loads the weights
#p = checkpointdir+modelname+"-training/cp-"

#fill in the best model filename like 0004
#best_checkpoint_path=p+""+"".ckpt"
#latest = tf.train.latest_checkpoint(checkpoint_dir)
#model.load_weights(latest)

# Re-evaluate the model
#loss, acc = model.evaluate(validation_images, verbose=2)
#print("Restored model, accuracy: {:5.2f}%".format(100 * acc))